### ![](../../img/installation-ico.png) Installation 

In [ ]:
pip install langchain langchain-community langchain-google-vertexai google.auth

Fonctionnalités nécessaires à la récupération de données web (scraping)

In [ ]:
pip install beautifulsoup4

### ![](../../img/package-ico.png) Imports

In [ ]:
import os
import google.auth

from langchain_google_vertexai import VertexAI

# pour la récupération de données depuis une page web
from langchain_community.document_loaders import WebBaseLoader

# pour le templeting des messages du prompt d'entrée
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain.prompts import HumanMessagePromptTemplate

# pour le parser de sortie
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

### ![](../../img/parametrage-ico.png) Paramétrages 

In [ ]:
# Gestion credentials VertexAI

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../settings/credentials.json"
google.auth.default()

In [ ]:
# Étape 1 : Charger le document avec un DataLoader
url = "https://newsroom.sfeir.com/mascotte-sfeir-pourquoi-avoir-choisi-un-axolotl-"
loader = WebBaseLoader(url)
documents = loader.load()
document_content = documents[0].page_content

print(document_content)

In [ ]:
# Étape 2 : Définir le modèle de réponse avec Pydantic
class Response(BaseModel):
    answer: str = Field(description="Answer to the question")


# Étape 3 : Définir le JsonOutputParser
parser = JsonOutputParser(pydantic_object=Response)


# Étape 4 : Définir le LLM
llm = VertexAI()

In [ ]:
# Étape 5 : Définir le ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        # Message système pour spécifier le comportement
        SystemMessage(content=("You are a helpful assistant.")),
        # Message système précisant le format de sortie
        SystemMessage(content=(parser.get_format_instructions())),
        # Message humain à partir d'un template
        HumanMessagePromptTemplate.from_template(
            """Answer the question based only on the following context:
            {context}

            Question: {question}
            """
        ),
    ]
)

In [ ]:
# Étape 6 : Chaîner les éléments
chain = prompt | llm | parser

In [ ]:
# Invoquer la chaîne en spécifiant le contexte et la question
result = chain.invoke(
    {
        "context": documents,
        "question": "Résumes moi ce document en une seule phrase",
    }
)
print(result)